# 1. Detect

We load our model via `torch.hub.load`, with a custom path that is the weights from our trained model for this project.
We also append our model confidence level threshold via `model.conf`, which is set to a balance between accuracy and conservativeness.

We will run the watchdog to track any changes happening in our file directory, which is a shared folder via Samba by the RPi.



In [1]:
# Model
import torch
import utils
model = torch.hub.load('.', 'custom', path='best.pt', source="local")
#model = torch.hub.load('ultralytics/yolov5', 'custom', path = 'best.pt')
model.conf = 0.5  # or yolov5n - yolov5x6, custom
model.max_det = 1

c:\Users\jakec\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
YOLOv5  2022-10-4 Python-3.10.6 torch-1.12.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


`on_modified` is the code we want to execute as a new file is created within the folder of our choice. In this case, we take the path of the new file created, and run object inference on the image. The results (class number and image with bounding box) is saved into the destination folder.

`path` lets the observer know which folder to track.

At this point, the observer is run and polling every half a second, for any new files created in the folder of our choice.

In [12]:
#create the watchdog to monitor file system changes
import time
from watchdog.observers import Observer
from watchdog.events import PatternMatchingEventHandler
import os
import cv2
import numpy as np

obstacles = 3

if __name__ == "__main__":
    
    patterns = ["*"]
    ignore_patterns = None
    ignore_directories = True
    case_sensitive = True
    my_event_handler = PatternMatchingEventHandler(patterns, ignore_patterns, ignore_directories, case_sensitive)
    
def on_modified(event):
    global incremental
        # Results
    try:
        
        global imgNo
        print(f"hey, {event.src_path} has been created!")
        time.sleep(0.3)
        # Images
        img2 = event.src_path


    # Inference
        results = model(img2)
        os.remove(img2)
        #time.sleep(0.2)
        results.print()
        #print("before save")
        results.save()
        #print("save image")
        incremental += 1


        df = results.pandas().xyxy[0]
        classNo = df._get_value(0, 'class')
        with open("A:/outputs/Output {}.txt".format(imgNo), "w") as text_file:
            text_file.write(str(classNo))
            print('Wrote to file Output {}.txt'.format(imgNo))
            imgNo += 1
            #print ("incremental : {}".format(incremental))
    except FileNotFoundError as f:
        #print("file not found")
        pass
    except KeyError as k:
        classNo = 99
        imgNo += 1
        print ("incremental : {}".format(incremental))

my_event_handler.on_created = on_modified

imgNo = 1 # do not change
incremental = 0
path = "A:\inputs"
go_recursively = False
my_observer = Observer()
my_observer.schedule(my_event_handler, path, recursive=go_recursively)  

try:
    my_observer.start()
except Exception:
    print('sad')
try:
    while incremental != obstacles:
        #print(incremental)
        time.sleep(0.5)
    
    # collage the pics
    directory = 'A:/outputs/pics'
    count = 0
    print("_____________________________COLLAGE____________________________")
    for filename in os.listdir(directory):

        f = os.path.join(directory, filename)
        print(str(f))

        if(count==0):
            image1 = cv2.imread(str(f))
            image1 = cv2.resize(image1,(400,600))
            combine = image1
        if(count==1):
            image2 = cv2.imread(str(f))
            image2 = cv2.resize(image2,(400,600))
            combine = np.hstack([image1,image2])
        if(count==2):
            image3 = cv2.imread(str(f))
            image3 = cv2.resize(image3,(400,600))
            combine =  np.hstack([image1,image2,image3])
        # if(count==3):
        #     image4 = cv2.imread(str(f))
        #     image4 = cv2.resize(image4,(200,200))
        #     h1 = np.hstack([image1, image2])
        #     h2 = np.hstack([image3, image4])
        #     combine = np.vstack([h1,h2])
        # if(count==4):
        #     image5 = cv2.imread(str(f))
        #     image5 = cv2.resize(image5,(400,200))
        #     combine = np.vstack([h1,h2,image5])
        # if(count==5):
        #     image6 = cv2.imread(str(f))
        #     image6 = cv2.resize(image6,(200,200))
        #     image5 = cv2.resize(image5,(200,200))
        #     h1 = np.hstack([image1, image2, image3])
        #     h2 = np.hstack([image4, image5, image6])
        #     combine = np.vstack([h1,h2])
        # if(count==6):
        #     image7 = cv2.imread(str(f))
        #     image7 = cv2.resize(image7,(600,200))
        #     combine = np.vstack([h1,h2,image7])
        # if(count==7):
        #     image8 = cv2.imread(str(f))
        #     image8 = cv2.resize(image8,(300,200))
        #     image7 = cv2.resize(image7,(300,200))
        #     h3 = np.hstack([image7,image8])
        #     combine = np.vstack([h1,h2,h3])

        count += 1

    cv2.imwrite('A:/outputs/final.jpg', combine)
    cv2.imshow("Final Collage",combine)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


except KeyboardInterrupt:
    my_observer.stop()
    my_observer.join()


my_observer.stop()

hey, A:\inputs\img1.jpg has been created!


image 1/1: 720x1280 1 right
Speed: 266.7ms pre-process, 208.8ms inference, 0.0ms NMS per image at shape (1, 3, 384, 640)
Saved 1 image to A:\outputs\pics


Wrote to file Output 1.txt
hey, A:\inputs\img1.jpg has been created!
hey, A:\inputs\img2.jpg has been created!


Exception in thread Thread-23:
Traceback (most recent call last):
  File "c:\Users\jakec\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\jakec\AppData\Local\Programs\Python\Python310\lib\site-packages\watchdog\observers\api.py", line 205, in run
    self.dispatch_events(self.event_queue)
  File "c:\Users\jakec\AppData\Local\Programs\Python\Python310\lib\site-packages\watchdog\observers\api.py", line 381, in dispatch_events
    handler.dispatch(event)
  File "c:\Users\jakec\AppData\Local\Programs\Python\Python310\lib\site-packages\watchdog\events.py", line 403, in dispatch
    super().dispatch(event)
  File "c:\Users\jakec\AppData\Local\Programs\Python\Python310\lib\site-packages\watchdog\events.py", line 272, in dispatch
    {
  File "C:\Users\jakec\AppData\Local\Temp\ipykernel_29652\3677752317.py", line 33, in on_modified
PermissionError: [WinError 5] Access is denied: 'A:\\inputs\\img2.jpg'
